In [1]:
import math
import sys
import os
import pandas as pd
import numpy as np
# Add the parent directory to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from overfitting import Strategy

In [2]:
# Define Constants
INITIAL_CAPTIAL=100000
SYMBOL = 'BTC'
CSV_PATH = './csv/binance_futures_BTCUSDT_1h 2019-09-09-2024-08-29.csv'
START_TIME='2023-01-01 00:00:00'
END_TIME='2024-08-29 00:00:00'

In [3]:
def gann_swing_chart(high: np.array, low: np.array):
    is_up = None
    tmp_max = high[0]
    tmp_min = low[0]
    tmp_max_i = 0
    tmp_min_i = 0

    tops = []
    bottoms = []

    for i in range(1, len(high)):
        
        up_day = high[i] > high[i-1] and low[i] > low[i-1]
        down_day = high[i] < high[i-1] and low[i] < low[i-1]

        # Determine the trend
        if is_up == None:
            if up_day:
                is_up=True
            elif down_day:
                is_up=False

        if is_up == True: # the last extreme is a bottom
            if high[i] > tmp_max:
                # New high Update
                tmp_max = high[i]
                tmp_max_i = i
            elif down_day == True:
                # Trend reversed, record it
                # top[0] = confirmation index
                # top[1] = index
                # top[2] = price
                top = [i, tmp_max_i, tmp_max]
                tops.append(top)

                # Setup for next bottom
                is_up = False
                tmp_min = low[i]
                tmp_min_i = i

        else: # the last extreme is a top
            if low[i] < tmp_min:
                # New low update
                tmp_min = low[i]
                tmp_min_i = i
            elif up_day == True:
                # Trend reversed, record it
                # bottom[0] = confirmation index
                # bottom[1] = index
                # bottom[2] = price
                bottom = [i, tmp_min_i, tmp_min]
                bottoms.append(bottom)

                # Setup for next top
                is_up=True
                tmp_max = high[i]
                tmp_max_i = i

    return tops, bottoms

def load_data(csv_path, start_time, end_time):
    # Load the CSV file into DataFrame
    df = pd.read_csv(csv_path)
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df.set_index('open_time', inplace=True)
    # Drop unnecessary columns
    df = df.drop(['close_time', 'volume'], axis=1)
    
    # Convert start_time and end_time to datetime
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)
    # Filter the dataframe by the start and end time
    df = df.loc[start_time:end_time]
    
    # Convert the relevant columns to numpy arrays
    high = df['high'].values
    low = df['low'].values

    # Apply the Gann Swing Chart calculation
    tops, bottoms = gann_swing_chart(high, low)

    # Store the tops and bottoms in the dataframe
    df['swing_top'] = np.nan
    df['swing_bottom'] = np.nan

    for top in tops:
        # Allocate tops in the corresponding index
        df.at[df.index[top[1]+ 1], 'swing_top'] = top[2]

    for bottom in bottoms:
        # Allocate tops in the corresponding index
        df.at[df.index[bottom[1] + 1], 'swing_bottom'] = bottom[2]

    return df


# Example usage
backtest_data = load_data(CSV_PATH, START_TIME, END_TIME)
print(backtest_data.head())


                              open           high            low  \
open_time                                                          
2023-01-01 00:00:00 16537.50000000 16540.90000000 16504.00000000   
2023-01-01 01:00:00 16527.10000000 16554.30000000 16524.10000000   
2023-01-01 02:00:00 16550.50000000 16557.10000000 16534.80000000   
2023-01-01 03:00:00 16542.50000000 16542.50000000 16515.00000000   
2023-01-01 04:00:00 16529.20000000 16530.40000000 16508.80000000   

                             close      swing_top  swing_bottom  
open_time                                                        
2023-01-01 00:00:00 16527.00000000            NaN           NaN  
2023-01-01 01:00:00 16550.40000000            NaN           NaN  
2023-01-01 02:00:00 16542.40000000            NaN           NaN  
2023-01-01 03:00:00 16529.30000000 16557.10000000           NaN  
2023-01-01 04:00:00 16517.80000000            NaN           NaN  


In [4]:
class MyStrategy(Strategy):
    def init(self):
        # Performance Analysis
        self.ups = 0
        self.downs = 0
        self.ups_edge = 0
        self.downs_edge = 0
        self.total_ups = 0
        self.total_downs = 0

        # Variables for swing trend detection
        self.prev_swing_top = None
        self.current_swing_top = None
        self.prev_swing_bottom = None
        self.current_swing_bottom = None
        self.current_trend = None

    def next(self, i):
        price = self.data.open[i]
        prev_low = self.data.low[i-1]
        prev_high = self.data.high[i-1]

        swing_top = self.data.swing_top[i]
        swing_bottom = self.data.swing_bottom[i]
        time = pd.to_datetime(self.data.timestamp[i])

        # Detect new swing top and bottom
        if not pd.isna(swing_top):
            self.prev_swing_top = self.current_swing_top if self.current_swing_top else None
            self.current_swing_top = swing_top
            if self.current_trend == 'up':
                # Close the uptrend position when swing top is confirmed
                p = self.get_position()
                
                self.market_order(SYMBOL, abs(p.qty))

                self.current_trend = None

        if not pd.isna(swing_bottom):
            self.prev_swing_bottom = self.current_swing_bottom if self.current_swing_bottom else None
            self.current_swing_bottom = swing_bottom
            if self.current_trend == 'down':
                # Close the downtrend position when swing bottom is confirmed
                p = self.get_position()
                
                self.market_order(SYMBOL, abs(p.qty))
                self.current_trend = None

        # Look for new trends when not in a position
        if self.current_trend is None:
            size = self.broker.cash / price
            size = math.floor(size * 1000) / 1000 

            if self.current_swing_bottom and self.prev_swing_bottom and self.current_swing_bottom > self.prev_swing_bottom:
                # Check for the uptrend
                if prev_high > self.current_swing_top:
                    # Starts the uptrend, enter position
                    self.market_order(SYMBOL, size)
                    self.current_trend = 'up'
                    self.total_ups += 1

            elif self.current_swing_top and self.prev_swing_top and self.current_swing_top < self.prev_swing_top:
                # Check for the downtrend
                if prev_low < self.current_swing_bottom:
                    # Starts the downtrend, enter position
                    self.market_order(SYMBOL, -size)
                    self.current_trend = 'down'
                    self.total_downs += 1

In [5]:
strategy = MyStrategy(backtest_data)
print(strategy)
returns = strategy.run()

Strategy(initial_capital=1000000, commission_rate=0.0002, slippage_rate=0, balances=[], returns=[])
{'id': '93a3753e140c455e918ed7c33640c187', 'created_at': Timestamp('2023-01-01 17:00:00'), 'symbol': 'BTC', 'qty': 60.4, 'price': None, 'type': <Enum.market: 4>, '_status': <Enum.OPEN: 1>, 'stop_price': None, 'trailing_delta': None, 'is_triggered': False, 'reason': None}
Order qty: 60.4, High: 16589.4, Price: None


TypeError: '>' not supported between instances of 'float' and 'NoneType'